In [1]:
import os
import cv2
import re
import random
import numpy as np
import pandas as pd
# For Visualization
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
import gc
# For Image Augmentation
import albumentations as A
from PIL import Image,ImageDraw
from ast import literal_eval

# For parallel processing
from joblib import Parallel,delayed

# For Object Detection
import torch
import torchvision
from torchvision import transforms
from torch.utils.data import Dataset,DataLoader
from sklearn.model_selection import train_test_split
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator

In [2]:
def load_model(filepath):
    checkpoint = torch.load(filepath)
    model = checkpoint['model']
    model.load_state_dict(checkpoint['state_dict'])
    for parameter in model.parameters():
        parameter.requires_grad = False
    
    model.eval()
    
    return model

# Loading Test File
test=os.listdir('/kaggle/input/global-wheat-detection/test')

# Loading Model
model=load_model('../input/fasterrcnn-model-weights/fasterrcnn_resnet50_fpn.pth')

# Image Transformation
image_transforms=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],
                            [0.229,0.224,0.225])
])

# Final Prediciton Funciton for merging data
def format_prediction_string(boxes, scores):
    pred_strings = []
    for j in zip(scores, boxes):
        s=j[0][0]
        x=float(round(j[1][0]))
        y=float(round(j[1][1]))
        w=float(round(j[1][2]-j[1][0]))
        h=float(round(j[1][3]-j[1][1]))
        pred_strings.append(f"{s}, {x}, {y}, {w}, {h}")

    return " ".join(pred_strings)

/opt/conda/lib/python3.7/site-packages/torch/serialization.py:657: SourceChangeWarning: source code of class 'torch.nn.modules.conv.Conv2d' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)


# Prediction

In [3]:
model=model.to(torch.device('cuda'))
threshold=0.5
box=[]
with torch.no_grad():
    for img_name in test:
        img_path=os.path.join(f'/kaggle/input/global-wheat-detection/test/{img_name}')
        img=Image.open(img_path)

        img=image_transforms(img)
        img=img.unsqueeze(0)

        img=img.to(torch.device('cuda'))

        output=model(img)
        
        # Format Prediction
        boxes=output[0]['boxes']
        scores=output[0]['scores']
        
        boxes=boxes[scores>threshold]  # Valid Boxes
        scores=scores[scores>threshold] # Valid Scores
        
        boxes=boxes.cpu().detach().tolist()
        scores=scores.cpu().detach()
        scores=scores.reshape(len(scores),1)
        scores=scores.tolist()
        
        
        boxes=format_prediction_string(boxes,scores)
        box.append(boxes)
        
        
        del img,img_path
        gc.collect()

prediction=pd.DataFrame({'image_id':test,'PredictionString':box})

/opt/conda/lib/python3.7/site-packages/torch/nn/functional.py:2854: UserWarning: The default behavior for interpolate/upsample with float scale_factor will change in 1.6.0 to align with other frameworks/libraries, and use scale_factor directly, instead of relying on the computed output size. If you wish to keep the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor will change "
/opt/conda/conda-bld/pytorch_1591914880026/work/torch/csrc/utils/python_arg_parser.cpp:756: UserWarning: This overload of nonzero is deprecated:
	nonzero(Tensor input, *, Tensor out)
Consider using one of the following signatures instead:
	nonzero(Tensor input, *, bool as_tuple)


In [4]:
prediction['image_id']=prediction['image_id'].apply(lambda x:x.split('.')[0])

# Submission

In [5]:
prediction['PredictionString']=prediction["PredictionString"].apply(lambda x: ' '.join(x.split(',')))

In [6]:
prediction.to_csv('/kaggle/working/submission.csv',index=False)

In [7]:
!ls /kaggle/working

__notebook__.ipynb  submission.csv
